In [ ]:
!pip install -q transformers datasets sentencepiece accelerate tensorboard


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.9 MB/s eta 0:00:00


In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

In [6]:
from google.colab import files

# Upload required JSON files
uploaded = files.upload()

Saving train_data_10k.json to train_data_10k.json


In [4]:
from google.colab import files

# Upload required JSON files
uploaded = files.upload()

Saving test_data_1k.json to test_data_1k.json


In [5]:
from google.colab import files

# Upload required JSON files
uploaded = files.upload()

Saving eval_data_1k.json to eval_data_1k (1).json


In [7]:
import json
from datasets import Dataset

# Helper function to load a .jsonl (JSON Lines) file
def load_json_lines(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f]

# Load datasets using JSON Lines
train_data = load_json_lines("train_data_10k.json")
eval_data = load_json_lines("eval_data_1k.json")
test_data = load_json_lines("test_data_1k.json")

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
test_dataset = Dataset.from_list(test_data)

print("✅ Loaded datasets:")
print("Train:", len(train_dataset))
print("Eval:", len(eval_dataset))
print("Test:", len(test_dataset))

✅ Loaded datasets:
Train: 10000
Eval: 1000
Test: 1000


In [8]:
import pandas as pd
from datasets import Dataset

# ✅ Updated path
DATA_PATH = "train_data_10k.json"  # after upload, file is in /content by default

# Load data
df = pd.read_json(DATA_PATH, lines=True)

print(f"Number of samples: {len(df)}")
print("Sample data:")
print(df.head(2))

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)

Number of samples: 10000
Sample data:
                                                 PFD  \
0  (raw)(hex){1}(hex){2}(mix)<2(r)[{tout}(v)(prod...   
1  (raw)(v)(tank)(pp)(v)(r)<1[{bout}(v)(prod)]{to...   

                                                 PID  
0  (raw)(hex){1}(C){TC}_1(hex){2}(mix)<2(r)<_2[(C...  
1  (raw)(v)<_1(tank)[(C){LC}_1](pp)[(C){M}<_2](C)...  


In [ ]:


MODEL_NAME = "t5-small"
MAX_LENGTH = 256

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:

def preprocess_function(examples):
    inputs = [f"assign tag: {pfd}" for pfd in examples["PFD"]]
    targets = examples["PID"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

#tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch


print(f"Train samples: {len(train_dataset)}, Eval samples: {len(eval_dataset)}, Test samples: {len(test_dataset)}")

MODEL_NAME = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Train samples: 10000, Eval samples: 1000, Test samples: 1000


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:

training_args = TrainingArguments(
    output_dir="./t5_pid_model",
    eval_strategy="steps",
    eval_steps=20,
    logging_steps=50,
    save_steps=40,
    save_total_limit=2,
    learning_rate=0.00005,
    remove_unused_columns=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="tensorboard",  # Change to "tensorboard" if you want logs
    fp16=torch.cuda.is_available(),
)

In [ ]:
#Trainer
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Use tokenized datasets in Trainer:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)


trainer.train()

tokenizer.save_pretrained("./t5_pid_model")
model.save_pretrained("./t5_pid_model")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
20,No log,2.014016
40,No log,1.212923
60,2.426200,0.865145
80,2.426200,0.682422
100,0.986300,0.571532
120,0.986300,0.498898
140,0.986300,0.445939
160,0.677600,0.414755
180,0.677600,0.380884
200,0.559400,0.355029


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


In [10]:
tokenizer.save_pretrained("./t5_pid_model")
model.save_pretrained("./t5_pid_model")

NameError: name 'tokenizer' is not defined

In [19]:
import traceback


def pfd_to_pid(pfd_text):
    try:
        input_text = f"assign tag: {pfd_text}"
        # Tokenize input
        inputs = tokenizer.encode(
            input_text,
            return_tensors="pt",
            max_length=500,
            truncation=True,
            padding="max_length"
        ).to(device)

        # Generate output
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=2000,  # Output PID is usually short
                num_beams=3,
            )

        # Decode and return prediction
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    except Exception as e:
        print(" Inference failed:", traceback.format_exc())
        return ""

# ✅ Run a test
print("\n=== Inference Test ===")
sample_pfd = df.loc[5, "PFD"]
print(" Input PFD:", sample_pfd[:300], "...")  # Truncate for readability
print(" Predicted PID:", pfd_to_pid(sample_pfd))
print(" True PID:", df.loc[0, "PID"])


=== Inference Test ===
 Input PFD: (raw)(hex){1}(comp)(v)(mix)<&|(raw)(v)&|(mix)<&|(raw)(mix)<2(pp)(splt)[(v)2](v)&|(hex){2}(mix)<1(pp)(splt)[(r)(v)(hex){2}(prod)](v)1n|(raw)(hex){1}(v)(prod) ...
 Predicted PID: (raw)(hex)1(comp)(v)(mix)&|(raw)(mix)&|(raw)(mix)1(pp)(splt)[(r)(v)(hex)2(prod)](v)1n|(raw)(hex)1(v)(prod)](v)1n|(raw)(hex)1(prod)(pro
 True PID: (raw)(hex){1}(C){TC}_1(hex){2}(mix)<2(r)<_2[(C){TC}_2][(C){LC}_3][{tout}(C){PC}_4(v)<_4(prod)]{bout}(v)<_3(splt)[(hex){2}(hex){3}(C){TC}_5(pp)[(C){M}](C){PI}(C){FC}_6(v)<_6(mix)<1(r)<_7[(C){TC}_7][(C){LC}_8][{bout}(v)<_8(prod)]{tout}(C){PC}_9(v)<_9(splt)[(C){FC}_10(v)1<_10](hex){4}(r)<_11[(C){TC}_11][(C){LC}_12][{tout}(C){PC}_13(v)<_13(prod)]{bout}(v)<_12(hex){4}(prod)](C){FC}_14(v)2<_14n|(raw)(hex){1}(v)<_1(prod)n|(raw)(hex){3}(v)<_5(prod)


In [22]:
from tqdm import tqdm

# Make predictions for all PFDs
predicted_pids = []
df = pd.read_json("test_data_1k.json", lines=True)
for pfd in tqdm(df["PFD"], desc="Predicting"):
    predicted_pid = pfd_to_pid(pfd)
    predicted_pids.append(predicted_pid)

Predicting: 100%|██████████| 1000/1000 [21:23<00:00,  1.28s/it]


In [23]:
# Add predictions to the DataFrame
df["Predicted_PID"] = predicted_pids

# Strip whitespace
df["PID"] = df["PID"].str.strip()
df["Predicted_PID"] = df["Predicted_PID"].str.strip()

# Accuracy: exact match
accuracy = (df["PID"] == df["Predicted_PID"]).mean()
print(f"\n✅ Accuracy: {accuracy:.4f}")


✅ Accuracy: 0.0000


In [24]:
from datasets import load_metric
from tqdm import tqdm

# Load metrics
bleu = load_metric("bleu")
rouge = load_metric("rouge")
bertscore = load_metric("bertscore")

# Lists for predictions and references
preds = []
refs = []

for pfd_text, true_pid_text in tqdm(zip(df["PFD"], df["PID"]), total=len(df)):
    pred = pfd_to_pid(pfd_text)
    preds.append(pred.strip())
    refs.append(true_pid_text.strip())

# Format for BLEU (needs list of tokens)
bleu_score = bleu.compute(predictions=[p.split() for p in preds], references=[[r.split()] for r in refs])
rouge_score = rouge.compute(predictions=preds, references=refs)
bert_score = bertscore.compute(predictions=preds, references=refs, lang="en")

# Print results
print(f"\n✅ BLEU: {bleu_score['bleu']:.4f}")
print(f"✅ ROUGE-L: {rouge_score['rougeL']:.4f}")
print(f"✅ BERTScore F1: {sum(bert_score['f1']) / len(bert_score['f1']):.4f}")

<ipython-input-24-c85c77699c3e>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("bleu")


ImportError: To be able to use rouge, you need to install the following dependency: rouge_score.
Please install it using 'pip install rouge_score' for instance.